In [78]:
import os

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

%matplotlib inline

In [2]:
import utilities

In [3]:
from importlib import reload

In [4]:
reload(utilities)

<module 'utilities' from 'C:\\Users\\luisr\\SharedProjects\\HowToWinKaggle\\FinalProject\\utilities.py'>

In [5]:
sales = utilities.load_and_preprocess()

In [6]:
agg_sales = utilities.aggregate_monthly(sales)

In [43]:
piv_agg_sales = agg_sales.set_index(['Month', 'shop_id', 'item_id']).item_cnt_month.unstack(level=0).fillna(0)

In [41]:
def create_windowed_XY(df_, fit_cols_):
    total_pred_months = df_.shape[1] - fit_cols_
    if total_pred_months <= 0:
        return (None, None)
    
    Xs = []
    Ys = []
    for i in range(0,total_pred_months):
        Xs.append(df_.iloc[:,i:fit_cols_+i].values)
        Ys.append(df_.iloc[:,fit_cols_+i].values)

    Y = np.hstack(Ys)
    X = np.row_stack(Xs)
    return X,Y
    

In [ ]:
def cross_val_score(df, model, fit_window=24, strides=12, feat_gen_func_dict=None):
    
    
    

In [58]:
model = RandomForestRegressor(n_estimators=50, max_depth=4)

In [89]:
df = piv_agg_sales
fit_months = 24
strides = 3

In [90]:
%%time
total_months = df.shape[1]
fold_size = fit_months + strides + 1

in_sample_errors = []
validate_errors = []
for i in range(total_months - fold_size):
    print(f'working on iteration: {i}')
    sub_df = df.iloc[:, i:i+fold_size]
    X_train, Y_train = create_windowed_XY(sub_df.iloc[:,:-1], fit_months)
    X_validate, Y_validate = sub_df.iloc[:,-1-fit_months:-1].values, sub_df.iloc[:,-1].values

    fit_model = model.fit(X_train, Y_train)

    Y_train_pred = fit_model.predict(X_train)
    in_sample_error = mean_squared_error(Y_train, Y_train_pred)
    Y_validate_pred = fit_model.predict(X_validate)
    valid_error = mean_squared_error(Y_validate, Y_validate_pred)
    
    in_sample_errors.append(in_sample_error)
    validate_errors.append(valid_error)

working on iteration: 0
working on iteration: 1
working on iteration: 2
working on iteration: 3
working on iteration: 4
working on iteration: 5
Wall time: 5min 8s


In [91]:
in_sample_errors

[2.2562731559064133,
 4.137067135326525,
 5.0561025666971355,
 4.801660417929613,
 1.5422874475107948,
 0.6178976425004049]

In [92]:
validate_errors

[11.838788687574148,
 5.486727463769595,
 0.9113489343779012,
 0.7130311163959843,
 0.8717189563109937,
 21.29419041692999]

In [87]:
in_sample_errors

[0.2370578840199766,
 0.7811146245053546,
 1.0190072903758782,
 1.016142478431722,
 0.42676193893518466,
 0.1836083240505933]

In [88]:
validate_errors

[2.0168786758030763,
 0.9612326153195412,
 0.25560382185582425,
 0.1668423596753963,
 0.26633993335218814,
 0.7148568273809066]

## TODO
* create score from rolling fits
* add shop 'bouginess' score
* add which month is this feature